In [1]:
import pandas as pd

In [3]:
data_raw = pd.read_csv('flights.csv')[[
    'MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE',
    'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
    'DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'CANCELLED'
    ]]

airports = pd.read_csv('us-airports.csv')[['iata_code', 'local_region', 'latitude_deg', 'longitude_deg']]

ids = pd.read_csv('airport_id.csv')
ids['Description'] = ids['Description'].apply(lambda s: s.split(':')[0].split(', ')[-1])

/var/folders/87/w9wc8z8x29vckh_y2y97hdg00000gn/T/ipykernel_27567/2531405513.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data_raw = pd.read_csv('flights.csv')[[


In [4]:
data_raw

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY,CANCELLED
0,1,1,4,5,AS,ANC,SEA,-11.0,-22.0,0
1,1,1,4,10,AA,LAX,PBI,-8.0,-9.0,0
2,1,1,4,20,US,SFO,CLT,-2.0,5.0,0
3,1,1,4,20,AA,LAX,MIA,-5.0,-9.0,0
4,1,1,4,25,AS,SEA,ANC,-1.0,-21.0,0
...,...,...,...,...,...,...,...,...,...,...
5819074,12,31,4,2359,B6,LAX,BOS,-4.0,-26.0,0
5819075,12,31,4,2359,B6,JFK,PSE,-4.0,-16.0,0
5819076,12,31,4,2359,B6,JFK,SJU,-9.0,-8.0,0
5819077,12,31,4,2359,B6,MCO,SJU,-6.0,-10.0,0


In [5]:
data_merged = data_raw\
    .merge(airports, how='left', left_on='ORIGIN_AIRPORT', right_on='iata_code')\
    .merge(airports, how='left', left_on='DESTINATION_AIRPORT', right_on='iata_code')\
    .merge(ids, how='left', left_on='ORIGIN_AIRPORT', right_on='Code')\
    .merge(ids, how='left', left_on='DESTINATION_AIRPORT', right_on='Code')

In [6]:
data = data_merged.copy()
data['ORIGIN_STATE'] = data_merged['local_region_x'].fillna(data_merged['Description_x'])
data['DESTINATION_STATE'] = data_merged['local_region_y'].fillna(data_merged['Description_y'])

data = data[['MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE',
                 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ORIGIN_STATE', 'DESTINATION_STATE',
                 'DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'CANCELLED'
                 ]]

In [7]:
data_cleaned = data[
    (data['ORIGIN_STATE'] != 'PR') & (data['ORIGIN_STATE'] != 'TT') & (data['ORIGIN_STATE'] != 'VI') & (data['ORIGIN_STATE'].notna()) &\
    (data['DESTINATION_STATE'] != 'PR') & (data['DESTINATION_STATE'] != 'TT') & (data['DESTINATION_STATE'] != 'VI') & (data['DESTINATION_STATE'].notna()) &\
    ((data['CANCELLED'] == 1) | (data['ARRIVAL_DELAY'].notna()))
    ]

In [8]:
data_cleaned.loc[2676357]

MONTH                     6
DAY                      18
DAY_OF_WEEK               4
SCHEDULED_DEPARTURE    1000
AIRLINE                  HA
ORIGIN_AIRPORT          SAN
DESTINATION_AIRPORT     HNL
ORIGIN_STATE             CA
DESTINATION_STATE        HI
DEPARTURE_DELAY        -4.0
ARRIVAL_DELAY          22.0
CANCELLED                 0
Name: 2676357, dtype: object

In [10]:
data_cleaned = data_cleaned[['MONTH', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ORIGIN_STATE', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]
data_cleaned

,MONTH,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,ORIGIN_STATE,DEPARTURE_DELAY,ARRIVAL_DELAY
0,1,AS,ANC,SEA,AK,-11.0,-22.0
1,1,AA,LAX,PBI,CA,-8.0,-9.0
2,1,US,SFO,CLT,CA,-2.0,5.0
3,1,AA,LAX,MIA,CA,-5.0,-9.0
4,1,AS,SEA,ANC,WA,-1.0,-21.0
...,...,...,...,...,...,...,...
5819070,12,B6,RNO,JFK,NV,0.0,-21.0
5819071,12,B6,SLC,MCO,UT,16.0,17.0
5819072,12,B6,DEN,JFK,CO,7.0,-11.0
5819073,12,B6,ABQ,JFK,NM,16.0,3.0


In [12]:
data_cleaned.to_csv('flight_cleaned.csv')

In [32]:
airport_data = pd.read_csv('us-airports.csv')[[
    'iata_code', 'name', 'longitude_deg', 'latitude_deg'
    ]]

In [56]:
#busy_airports = ['ATL', 'LAX', 'ORD', 'DFW', 'JFK', 'DEN', 'SFO', 'CLT', 'LAS', 'PHX']
busy_airports = ['ATL', 'LAX', 'ORD', 'DFW', 'JFK', 'DEN']
airport_data_cleaned = airport_data[airport_data['iata_code'].isin(busy_airports)].reset_index(drop=True)
airport_data_cleaned

,iata_code,name,longitude_deg,latitude_deg,municipality,region_name
0,LAX,Los Angeles / Tom Bradley International Airport,-118.407997,33.942501,Los Angeles,California
1,ORD,Chicago O'Hare International Airport,-87.9048,41.9786,Chicago,Illinois
2,JFK,John F Kennedy International Airport,-73.779317,40.639447,New York,New York
3,ATL,Hartsfield Jackson Atlanta International Airport,-84.428101,33.6367,Atlanta,Georgia
4,DFW,Dallas Fort Worth International Airport,-97.038002,32.896801,Dallas-Fort Worth,Texas
5,DEN,Denver International Airport,-104.672996521,39.861698150635,Denver,Colorado


In [57]:
#airport_data_cleaned.to_csv('airport_data_cleaned.csv')